In [3]:
import numpy as np
import rigidpy as rp
from scipy.spatial.distance import pdist, squareform
import copy
import os

# VISUALIZATION
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
from matplotlib.patches import Rectangle, Circle
from scipy.linalg import eigh

In [4]:
# Returns coordinates of all unit and image nodes, given unit nodes
def unit_and_image_nodes(pos, Lx, Ly):
    N = len(pos)   # number of nodes
    
    # unit cell shifts for image cells
    top_left_shift = np.ones((N, 2)) * [-Lx, Ly]
    top_shift = np.ones((N, 2)) * [0, Ly]
    top_right_shift = np.ones((N, 2)) * [Lx, Ly]
    left_shift = np.ones((N, 2)) * [-Lx, 0]
    right_shift = np.ones((N, 2)) * [Lx, 0]
    bottom_left_shift = np.ones((N, 2)) * [-Lx, -Ly]
    bottom_shift = np.ones((N, 2)) * [0, -Ly]
    bottom_right_shift = np.ones((N, 2)) * [Lx, -Ly]

    # positions of all particles in unit cell and image cells
    pos_all = np.vstack((pos, pos+top_left_shift, pos+top_shift, pos + top_right_shift,\
                         pos + left_shift, pos + right_shift, \
                         pos+bottom_left_shift, pos+bottom_shift, pos + bottom_right_shift))
    
    return pos_all

In [5]:
def get_periodic_bonds(nodes, bonds, Lx, Ly):
    nodes_all = unit_and_image_nodes(nodes, Lx, Ly)
    periodic_bonds = []
    N = len(nodes)
    bond_id = []
    for i in range(len(bonds)):
        
        # additional bonds are the bonds in the 8 image nodes
        additional_bonds = np.zeros((16, 2), dtype = int)
        
        # fill in unit bonds for node 1 and 2
        additional_bonds.T[0][0:8] = bonds[i][0]
        additional_bonds.T[1][8:] = bonds[i][1]
        
        # add additional image bonds (those connected to node 1 and node 2)
        additional_bonds.T[1][0:8] = bonds[i][1] + np.arange(N, 9*N, N)
        additional_bonds.T[0][8:] = bonds[i][0] + np.arange(N, 9*N, N)
        
        # have additonal bond with original bond at beginning and flipped original bond at the end
        all_bonds = np.vstack((bonds[i], additional_bonds, np.array([bonds[i][1], bonds[i][0]])))
        
        # calculate bond lengths of each othese bonds
        all_lengths = bond_lengths(nodes_all, all_bonds)
        
        # have the 1st plotting bond be the shortest orignal bond and all bonds connected to node 1
        periodic_bond_1 = all_bonds[0:9][np.argmin(all_lengths[0:9])]
        
        # have the 2nd plotting bond be the shortest of all bonds connected to node2 with the flipped original bond
        periodic_bond_2 = all_bonds[9:][np.argmin(all_lengths[9:])]
        
        # if any of the plotting bonds (shortest bonds) dont share a node plot them both
        # and id them as the same bond
        if np.any(periodic_bond_1 != np.array([periodic_bond_2[1], periodic_bond_2[0]])):
            periodic_bonds.append(periodic_bond_1)
            periodic_bonds.append(periodic_bond_2)
            bond_id.append(i)
            bond_id.append(i)
        # if any of the plotting bonds share a node, then only plot it once and id as one bond
        else:
            periodic_bonds.append(periodic_bond_1)
            bond_id.append(i)
    
    # return the plotting bonds and bond_ids with all nodes (bonds sorted with lower node index first)
    periodic_bonds = np.array(periodic_bonds)
    periodic_bonds = np.sort(periodic_bonds, axis = 1)
    bond_id = np.array(bond_id, dtype = int)
    return nodes_all, periodic_bonds, bond_id

In [6]:
def plot(nodes, bonds, Lx, Ly, ks, stresses = np.array([])):
    
    # if I don't give any stresses, just make all bonds gray
    if len(stresses) == 0:
        stresses = np.zeros(len(bonds))
    
    # set up figure
    fig, ax = plt.subplots(figsize = (5, 5))
    
    # scatter the nodes
    plt.scatter(nodes.T[0], nodes.T[1], color = 'dimgrey', s = 20, zorder = 1)
    
    # color scale for the bond stresses
    color_norm  = colors.Normalize(vmin=-1, vmax=1)
    scalar_map = cmx.ScalarMappable(norm=color_norm, cmap="coolwarm")
    
    # plot periodic bonds
    nodes_all, plotting_bonds, bond_id = get_periodic_bonds(nodes, bonds, Lx, Ly)
    
    # loop through each bond and plot it as a line with the correct color
    for i in range(len(plotting_bonds)):
        p = nodes_all[plotting_bonds[i]]
        x1 = p[0][0]
        y1 = p[0][1]
        x2 = p[1][0]
        y2 = p[1][1]
        color_val = scalar_map.to_rgba(stresses[bond_id[i]])
        if ks[bond_id[i]] > 0:
            plt.plot([x1, x2], [y1, y2], color = color_val, linewidth = 4, zorder=0)
    
    # format the axis
    plt.axis('square')
    plt.xlim(0, Lx)
    plt.ylim(0, Ly)

    plt.gca().add_patch(Rectangle((0, 0), Lx, Ly, linewidth=6, edgecolor='black', facecolor='none', linestyle = 'dashed'))
    plt.axis('off')

In [7]:
# open packing file and get positions, radii, and box size
def get_packing(network_size, packing_pressure, packing_id):
    
    # open the packing file
    packing_file = '2dPackings_N' + str(network_size) + '_dphi' + packing_pressure + '/' + str(packing_id) + '/'
    
    # identify the stable nodes to reassign node indices eliminating unstable nodes
    stable_list = np.loadtxt(packing_file + 'stableList.dat', dtype = int)
    unstable_nodes = np.arange(len(stable_list))[stable_list == 0]
    
    # find node index shifts after removing nodes
    reassignment_shift = np.zeros(len(stable_list), dtype = int)
    if len(unstable_nodes) > 0:
        for i in range(len(unstable_nodes) - 1):
            reassignment_shift[unstable_nodes[i] : unstable_nodes[i + 1]] = i + 1
        reassignment_shift[unstable_nodes[-1]:] = len(unstable_nodes)
        
    # read in positions and radii only of stable nodes
    pos = np.loadtxt(packing_file + 'positions.dat', dtype = np.float64)[stable_list == 1]
    rs = np.loadtxt(packing_file + 'radii.dat', dtype = np.float64)[stable_list == 1]
    
    # read in bonds and reassign node indices
    fs_sparse = np.loadtxt(packing_file + 'forces.dat', skiprows = 3)
    bonds = np.array(fs_sparse.T[0:2].T, dtype = int) - 1
    bonds = bonds[np.logical_not(np.any(np.isin(bonds, unstable_nodes), axis = 1))]
    bonds = bonds - reassignment_shift[bonds]
    
    # read out forces and box size
    fs = np.array(fs_sparse.T[2], dtype = np.float64)
    box_size = np.loadtxt(packing_file + 'boxSize.dat', dtype = np.float64)
    Lx = box_size[0]
    Ly = box_size[1]
    
    return pos, bonds, fs, Lx, Ly, rs

In [8]:
def plot_packing(network_size, packing_pressure, packing_id):
    pos, bonds, fs, Lx, Ly, rs = get_packing(network_size, packing_pressure, packing_id)
    pos_all = unit_and_image_nodes(pos, Lx, Ly)
    radii = np.tile(rs, 9)
    N = len(pos)
    
    Aforce = np.zeros((N, N))
    Aforce[bonds.T[0], bonds.T[1]] = fs
    Aforce[bonds.T[1], bonds.T[0]] = fs
    Aforce = np.array(Aforce, dtype = bool)
    
    f_particles = np.sum(Aforce, axis = 1)
    f_particles_all = np.tile(f_particles, 9)
    
    color_norm  = colors.Normalize(vmin=0, vmax=np.max(f_particles_all))
    scalar_map = cmx.ScalarMappable(norm=color_norm, cmap="Blues")
    
    for i in range(len(pos_all)):
        color = scalar_map.to_rgba(f_particles_all[i])
        plt.gca().add_patch(Circle((pos_all[i].T[0], pos_all[i].T[1]), radii[i], edgecolor = 'black', facecolor = color, alpha = 0.75))

    plt.axis('square')
    plt.axis('off')
    plt.xlim(0, 1)
    plt.ylim(0, 1)

In [9]:
def bond_lengths(nodes, bonds):
    node1_pos = nodes[bonds.T[0]]
    node2_pos = nodes[bonds.T[1]]

    lengths = np.sqrt((node2_pos.T[0] - node1_pos.T[0])**2 + (node2_pos.T[1] - node1_pos.T[1])**2)
    return lengths

In [13]:
# get all the bonds of the system and overlaps
def get_network(network_size, packing_pressure, packing_id):
    pos, bonds, fs, Lx, Ly, rs = get_packing(network_size, packing_pressure, packing_id)
    if len(pos) == 0:
        return None
    
    basis = np.array([[Lx, 0], [0, Ly]])
    F = rp.framework(pos, bonds, basis)
    rij = F.edgeLengths()
    sij = rs[bonds.T[0]] + rs[bonds.T[1]]
    overlaps = sij - rij
    network = Network(pos, bonds, Lx, Ly, dl0s = overlaps, ks =  1/sij**2)
    return network

In [14]:
class Network:
    
    def __init__(self, nodes, bonds, Lx=1, Ly=1, dl0s = np.array([]), l0s = None, ks = None, basis = None):
        self.nodes = np.array(nodes, dtype = np.float64)
        self.bonds = bonds
        self.dl0s = dl0s
        self.Lx = np.float64(Lx)
        self.Ly = np.float64(Ly)
        if basis is None:
            self.basis = np.array([[self.Lx, 0], [0, self.Ly]])
        else:
            self.basis = basis
        
        if ks is None:
            self.ks = np.ones(len(bonds))
        else:
            self.ks = ks
            
        self.F = rp.framework(self.nodes, self.bonds, self.basis, k=self.ks)
        
        if l0s is None:
            self.l0s = self.F.L0
        else:
            self.l0s = l0s 
    
    # makes a secure copy of the network
    def copy(self):
        net = copy.deepcopy(self)
        return net
    
    # creates adjacency matrix for the network
    def A(self):
        N = len(self.nodes)
        Amat = np.zeros((N, N))
        current_bonds = self.bonds[self.ks > 0]
        Amat[current_bonds.T[0], current_bonds.T[1]] = 1
        Amat[current_bonds.T[1], current_bonds.T[0]] = 1
        Amat = np.array(Amat, dtype = bool)
        return Amat 
    
    # returns the connectivity of the matrix
    def dZ(self):
        Nb = len(self.bonds)
        N = len(self.nodes)
        Z = 2*Nb/N
        return Z - 4
     
    # plots the matrix
    def plot(self):
        stresses = self.stresses()
        plot(self.nodes, self.bonds, self.Lx, self.Ly, self.ks, stresses)
    
    # calculates network stresses
    def stresses(self):
        lengths = self.F.edgeLengths()
        fs = self.ks * (lengths - self.l0s)
        return fs
    
    # calculates bond extensions
    def dr(self):
        lengths = self.F.edgeLengths()
        return lengths - self.l0s
    
    # calculate bond strains
    def strains(self):
        return self.dr() / self.l0s
    
    # calculates energy in the network
    def energy(self):
        return np.sum(0.5 * self.ks * self.dr()**2)
    
    # returns rigidity matrix of the network
    def rigidity_matrix(self):
        F = rp.framework(self.nodes, self.bonds, self.basis, k = self.ks, restLengths = self.l0s)
        return F.rigidityMatrix()
    
    # returns hessian of the network
    def hessian_matrix(self):
        F = rp.framework(coordinates = self.nodes, bonds = self.bonds, basis = self.basis, k = self.ks, restLengths = self.l0s)
        return F.hessianMatrix()
    
    # returns prestress matrix of the network
    def prestress_matrix(self):
        F = rp.framework(self.nodes, self.bonds, self.basis, k = self.ks, restLengths = self.l0s)
        return F.hessianMatrixPrestress()
    
    # returns geometric hessian of the network
    def geometric_matrix(self):
        F = rp.framework(self.nodes, self.bonds, self.basis, k = self.ks, restLengths = self.l0s)
        return F.hessianMatrixGeometric()
    
    # returns states of self stress of the network
    def states_of_self_stress(self):
        F = rp.framework(self.nodes, self.bonds, self.basis, k = self.ks, restLengths = self.l0s)
        return F.selfStress().T
    
    # relaxes the network
    def relax(self, new_nodes = None, new_ks = None, new_l0s = None, new_Lx = None, new_Ly = None, new_basis = None):
        if new_nodes is None:
            new_nodes = self.nodes
        if new_ks is None:
            new_ks = self.ks
        if new_l0s is None:
            new_l0s = self.l0s
        if new_Lx is None:
            new_Lx = self.Lx
        if new_Ly is None:
            new_Ly = self.Ly
        if new_basis is None:
            new_basis = np.array([[new_Lx, 0], [0, new_Ly]])
              
        config = rp.configuration(new_nodes, self.bonds, basis = new_basis, k=new_ks)
        relaxed_coords = config.energyMinimizeNewton(new_l0s, new_l0s)
        relaxed_network = Network(relaxed_coords, self.bonds, basis = new_basis, dl0s = self.dl0s, ks=new_ks, l0s=new_l0s)
        return relaxed_network
    
    # applies bulk deformation
    def bulk(self, s):
        gamma = 1 + s
        relaxed_network = self.relax(new_Lx=self.Lx*gamma, new_Ly=self.Ly*gamma)
        return relaxed_network
    
    # applies shear deformation
    def shear(self, s):
        gamma = 1 + s
        relaxed_network = self.relax(new_Lx=self.Lx/gamma, new_Ly=self.Ly*gamma)
        return relaxed_network
    
    # finds normal modes of the network
    def normal_modes(self):
        F = rp.framework(self.nodes, self.bonds, basis = self.basis, k = self.ks, restLengths = self.l0s)
        eigenvals, eigenvecs = F.eigenSpace(eigvals=None)
        N = len(self.nodes)
        eigenvecs = eigenvecs.reshape(2*N, N, 2)
        return eigenvals, eigenvecs
    
    # bulk modulus
    def B(self, eps):
        F = rp.framework(self.nodes, self.bonds, self.basis, restLengths=self.l0s, k=self.ks)
        bulk_mod = F.bulkModulus(eps = eps)
        return bulk_mod
    
    # shear modulus    
    def G(self, eps):
        F = rp.framework(self.nodes, self.bonds, self.basis, restLengths=self.l0s, k=self.ks)
        shear_mod = F.shearModulus(eps = eps)
        return shear_mod
    
    # prunes a bond in the network
    def prune(self, bond_ind):
        new_ks = np.ones(self.ks.shape)
        np.copyto(new_ks, self.ks)
        new_ks[bond_ind] = 0
        relaxed_network = self.relax(new_ks=new_ks)
        return relaxed_network
     
    # applies shear deformation and calculates shear modulus
    def shear_modulus(self, eps):
        mat = np.array([[1+eps, 0], [0, 1 / (1 + eps)]])
        new_nodes = np.matmul(mat, self.nodes.T).T
        ls = np.matmul(mat, np.array([[self.Lx], [self.Ly]])).T.flatten()
        sheared_network = self.relax(new_nodes, new_Lx = ls[0], new_Ly = ls[1])
        F = rp.framework(self.nodes, self.bonds, self.basis, k = self.ks, 
                        restLengths = self.l0s)
        G = F.elasticModulus(mat)
        return sheared_network, G
    
    # applies bulk deformation and calculates bulk modulus
    def bulk_modulus(self, eps):
        mat = np.array([[1+eps, 0], [0, 1 + eps]])
        new_nodes = np.matmul(mat, self.nodes.T).T
        ls = np.matmul(mat, np.array([[self.Lx], [self.Ly]])).T.flatten()
        bulk_network = self.relax(new_nodes, new_Lx = ls[0], new_Ly = ls[1])
        F = rp.framework(self.nodes, self.bonds, self.basis, k = self.ks, 
                        restLengths = self.l0s)
        B = F.elasticModulus(mat)
        return bulk_network, B
    
    # saves network files
    def save(self, filename):
        os.mkdir(filename)
        np.save(filename + '/nodes.npy', self.nodes)
        np.save(filename + '/bonds.npy', self.bonds)
        np.save(filename + '/dl0s.npy', self.dl0s)
        np.save(filename + '/Lx.npy', self.Lx)
        np.save(filename + '/Ly.npy', self.Ly)
        np.save(filename + '/ks.npy', self.ks)
        np.save(filename + '/l0s.npy', self.l0s)
    

In [15]:
# loads network given network files
def load(filename):
    nodes = np.load(filename + '/nodes.npy')
    bonds = np.load(filename + '/bonds.npy')
    dl0s = np.load(filename + '/dl0s.npy')
    Lx = np.load(filename + '/Lx.npy')
    Ly = np.load(filename + '/Ly.npy')
    ks = np.load(filename + '/ks.npy')
    l0s = np.load(filename + '/l0s.npy')
    net = Network(nodes, bonds, Lx, Ly, dl0s = dl0s, l0s = l0s, ks = ks)
    return net